### 勾配法

広大なパラメータの空間から最適値を求めるのに勾配をうまく使って関数の最小値を求めるのが勾配法。
ニューラルネットワークの学習では勾配法がよく用いられる。

勾配法を数式で表すと次のようになる。

$$
x_0 = x_0 - \eta \frac{\partial f}{\partial x_0} \\
x_1 = x_1 - \eta \frac{\partial f}{\partial x_1}
$$

$\eta$は更新の量を表し、ニューラルネットワークの学習においては学習率と呼ばれる。

In [1]:
import numpy as np

def _numerical_gradient_no_batch(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 値を元に戻す
        
    return grad


def numerical_gradient(f, X):
    if X.ndim == 1:
        return _numerical_gradient_no_batch(f, X)
    else:
        grad = np.zeros_like(X)
        
        for idx, x in enumerate(X):
            grad[idx] = _numerical_gradient_no_batch(f, x)
        
        return grad

def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
        
    return x

In [2]:
# f(x0, x) = x0^2 + x1^2の最小値を勾配法で求める

def function_2(x):
    return x[0]**2 + x[1]**2

init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100)

array([ -6.11110793e-10,   8.14814391e-10])

In [6]:
# lrは大きすぎても小さすぎてもうまくいかない
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=10.0, step_num=100)

array([ -2.58983747e+13,  -1.29524862e+12])

In [8]:
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=0.001, step_num=100)

array([-2.45570041,  3.27426722])

> 学習率のようなパラメータはハイパーパラメータという。ニューラルネットワークの重みやバイアスのような自動で獲得されるものとは異なり、人の手によって設定される。このハイパーパラメータをいろいろ試してうまく学習できるようにする作業が必要になる。

### ニューラルネットワークでの勾配

重みパラメータに関する損失関数の勾配を求める。
重みパラメータの行列と勾配の行列は同じ形になる。

In [10]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient


class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

In [11]:
net = simpleNet()
net.W

array([[-0.37064593, -0.05938241,  0.70608901],
       [-0.10274108, -0.92581385,  0.75432652]])

In [12]:
x = np.array([0.6, 0.9])
p = net.predict(x)
p

array([-0.31485453, -0.86886191,  1.10254728])

In [13]:
np.argmax(p)

2

In [15]:
t = np.array([0, 0, 1])
net.loss(x, t)

0.32324465476914266

In [17]:
def f(W):
    return net.loss(x, t)

dW = numerical_gradient(f, net.W)
dW

array([[ 0.10524418,  0.06047777, -0.16572195],
       [ 0.15786627,  0.09071665, -0.24858292]])

In [19]:
f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)
dW

array([[ 0.10524418,  0.06047777, -0.16572195],
       [ 0.15786627,  0.09071665, -0.24858292]])